In [1]:
from serial.tools.list_ports import comports

def arduino_port():
    for port in comports():
        if 'Arduino' in port.description:
            return port.name
    return None

port = arduino_port()
display(f'arduino contected: {port}')

'arduino contected: COM9'

In [2]:
from os import scandir
import numpy as np
import pandas as pd

def get_data(dataset_name, channels):    
    path = f'./data/{dataset_name}'
    
    with scandir(path) as it:
        entries = []
        for entry in it:
            if not entry.name.startswith('.') and entry.name.endswith('.csv') and entry.is_file():
                entries.append(entry)
    
    records = [None] * len(entries)
    targets = [None] * len(entries)
    
    for entry in entries:
        i = int(entry.name[:entry.name.index('_')])
        targets[i] = entry.name[entry.name.index('_')+1:entry.name.index('.')]
        records[i] = pd.read_csv(entry.path, usecols=channels)
    display('Data acquired from ' + dataset_name)
    return records, targets

records, targets = get_data('kaggle_dataset', ['emg0','emg1'])
display(targets)
display(len(records))
records[0]

'Data acquired from kaggle_dataset'

['neutral', 'flexion', 'extension', 'fist']

4

,emg0,emg1
0,19,26
1,20,26
2,20,23
3,19,24
4,18,24
...,...,...
83605,23,111
83606,21,95
83607,23,109
83608,22,109


In [3]:
from tqdm.auto import tqdm
from time import sleep, time

def seconds(): 
    return int(time())
def keep_runing(func, duration):
    results = []
    
    with tqdm(total=duration) as bar:
        
        current = seconds()
        end = current + duration
        
        while current < end:
            
            results.append(func())

            last = current
            current = seconds()
            bar.update(current - last)
    
    return np.array(results)

keep_runing(seconds, 1)

  0%|          | 0/1 [00:00<?, ?it/s]

array([1706784605, 1706784605, 1706784605, ..., 1706784605, 1706784605,
       1706784606])

In [6]:
from serial import Serial

def read_serial(serial, count):
    while True:
        try:
            values = serial.readline().decode('utf-8').strip().split()
        except UnicodeDecodeError:
            continue
        if len(values) == count:
            return [int(value) for value in values]
def capture(serial, count, target, duration, wait):
    
    display(f'Capturing for {target} will start in {wait} seconds')
    sleep(wait)
    record = keep_runing(lambda: read_serial(serial,count), duration)
    display(f'Captured {len(record)} data')

    return record

if port:
    with Serial(port, 9600, timeout=5) as serial:
        display(read_serial(serial,2))
        display(capture(serial, 2, 'flexion', 1, 1))

[422, 399]

'Capturing for flexion will start in 1 seconds'

  0%|          | 0/1 [00:00<?, ?it/s]

'Captured 168 data'

array([[328, 334],
       [352, 346],
       [333, 334],
       [319, 319],
       [373, 370],
       [304, 310],
       [419, 405],
       [289, 301],
       [429, 413],
       [280, 293],
       [425, 407],
       [271, 286],
       [409, 389],
       [265, 278],
       [357, 342],
       [262, 270],
       [295, 287],
       [266, 269],
       [258, 258],
       [304, 301],
       [245, 250],
       [354, 342],
       [231, 243],
       [370, 354],
       [224, 237],
       [370, 352],
       [217, 232],
       [362, 342],
       [214, 226],
       [314, 298],
       [212, 220],
       [255, 245],
       [215, 218],
       [216, 213],
       [249, 248],
       [201, 205],
       [302, 292],
       [189, 199],
       [326, 310],
       [183, 196],
       [330, 312],
       [179, 193],
       [327, 308],
       [177, 190],
       [287, 268],
       [177, 186],
       [229, 218],
       [179, 183],
       [188, 184],
       [210, 209],
       [171, 175],
       [264, 256],
       [162,

In [7]:
from serial import Serial

def capture_data(channels, targets, duration, wait, baud_rate):
    with Serial(arduino_port(), baud_rate, timeout=5) as serial:
        
        # discard first line to be sure new line is properly formatted
        serial.readline()
        serial.reset_input_buffer()

        records = []
        for target in targets:
            record = capture(serial, len(channels), target, duration, wait)
            df = pd.DataFrame(record, columns=channels)
            records.append(df)
    return records

if port:
    records = capture_data(['emg0','emg1'], ['neutral','flexion','extension'], 0.5, 0.5, 9600)
    display(len(records))
    display(records[0])

'Capturing for neutral will start in 0.5 seconds'

  0%|          | 0/0.5 [00:00<?, ?it/s]

'Captured 128 data'

'Capturing for flexion will start in 0.5 seconds'

  0%|          | 0/0.5 [00:00<?, ?it/s]

...

'Capturing for extension will start in 0.5 seconds'

  0%|          | 0/0.5 [00:00<?, ?it/s]

'Captured 96 data'

3

,emg0,emg1
0,335,335
1,339,335
2,358,358
3,316,320
4,407,399
...,...,...
123,153,148
124,158,159
125,134,136
126,215,208


In [8]:
from pathlib import Path
from IPython.display import FileLinks

def save_data(records, targets, name=None):
    
    if name is None:
        name = seconds()

    path = Path(f'./data/{name}')
    path.mkdir(exist_ok=True,)

    for i,target in enumerate(targets):
        target_path = path.joinpath(f'{i}_{target}.csv')
        records[i].to_csv(target_path, index=False)
    display(f'Data saved to {path}')

save_data(records, ['neutral','flexion','extension'], 'test')
FileLinks('./data/test/', recursive=False)

'Data saved to data\\test'

./data/test/
  0_neutral.csv
  0_relax.csv
  1_flex.csv
  1_flexion.csv
  2_extension.csv
  2_fist.csv

In [9]:
# number of sensor inputs
channel_count = 2
channels = [f'emg{i}' for i in range(channel_count)]
# gestures in use
targets = ['relax', 'flex', 'fist']
# dataset to use when arduino is not connected
dataset = 'kaggle_dataset'
# save captured data as when arduino is connected
save_as = 'test'

# recording time
duration = 30
wait = 5
baud_rate = 9600

if arduino_port() is None:
    records, targets = get_data(dataset, channels)
else:
    records = capture_data(channels, targets, duration, wait, baud_rate)
    save_data(records, targets, save_as)

'Capturing for relax will start in 5 seconds'

  0%|          | 0/30 [00:00<?, ?it/s]

'Captured 3252 data'

'Capturing for flex will start in 5 seconds'

  0%|          | 0/30 [00:00<?, ?it/s]

'Captured 3323 data'

'Capturing for fist will start in 5 seconds'

  0%|          | 0/30 [00:00<?, ?it/s]

'Captured 3321 data'

'Data saved to data\\test'

In [10]:
targets

['relax', 'flex', 'fist']

In [11]:
records

[      emg0  emg1
 0      467   438
 1      321   332
 2      404   389
 3      316   322
 4      341   335
 ...    ...   ...
 3247   122   136
 3248   266   249
 3249   123   137
 3250   262   241
 3251   127   138
 
 [3252 rows x 2 columns],
       emg0  emg1
 0      219   204
 1      130   138
 2      167   158
 3      140   143
 4      136   135
 ...    ...   ...
 3318   238   228
 3319   121   133
 3320   265   248
 3321   121   135
 3322   270   252
 
 [3323 rows x 2 columns],
       emg0  emg1
 0      122   136
 1      270   248
 2      125   137
 3      225   210
 4      129   137
 ...    ...   ...
 3316   259   244
 3317   121   135
 3318   270   252
 3319   122   135
 3320   272   253
 
 [3321 rows x 2 columns]]

In [12]:
!git clone https://github.com/SebastianRestrepoA/EMG-pattern-recognition.git emg_pattern_recognition

fatal: destination path 'emg_pattern_recognition' already exists and is not an empty directory.


In [13]:
from emg_pattern_recognition.feature_extraction import features_estimation

In [14]:
record = records[0]
record

,emg0,emg1
0,467,438
1,321,332
2,404,389
3,316,322
4,341,335
...,...,...
3247,122,136
3248,266,249
3249,123,137
3250,262,241


In [15]:
list(record.items())

[('emg0',
  0       467
  1       321
  2       404
  3       316
  4       341
         ... 
  3247    122
  3248    266
  3249    123
  3250    262
  3251    127
  Name: emg0, Length: 3252, dtype: int32),
 ('emg1',
  0       438
  1       332
  2       389
  3       322
  4       335
         ... 
  3247    136
  3248    249
  3249    137
  3250    241
  3251    138
  Name: emg1, Length: 3252, dtype: int32)]

In [16]:
features_dfs = [
    features_estimation(np.array(channel), channel_name, 10, 25, 10)[0]
    for channel_name,channel in record.items()
]
features_dfs[0]

C:\Users\LeNoVo\Desktop\31Project\v3\emg_pattern_recognition\feature_extraction.py:169: RuntimeWarning: invalid value encountered in scalar divide
  return ULC / UHC
C:\Users\LeNoVo\miniforge3\envs\myjlabenv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(


EMG features were from channel emg0 extracted successfully
EMG features were from channel emg1 extracted successfully


,0,1,2,3,4,5,6,7,8,9,...,313,314,315,316,317,318,319,320,321,322
VAR,3.978560e+03,4.307840e+03,3.712438e+03,3.346074e+03,3.836870e+03,3276.000000,3077.449600,3740.550400,3158.073600,3081.689600,...,2635.321600,3121.577600,3250.169600,2648.320000,3245.529600,3317.769600,2753.440000,3353.369600,3244.134400,2715.273600
RMS,3.306716e+02,2.981149e+02,2.640850e+02,2.401507e+02,2.279379e+02,208.798084,201.275831,201.848755,189.599684,189.780610,...,181.146239,183.397601,185.459861,181.258820,184.495962,184.729640,181.740034,185.737880,184.187839,181.711640
IEMG,8.115000e+03,7.270000e+03,6.424000e+03,5.827000e+03,5.484000e+03,5020.000000,4837.000000,4809.000000,4527.000000,4537.000000,...,4343.000000,4367.000000,4412.000000,4345.000000,4387.000000,4388.000000,4350.000000,4412.000000,4379.000000,4352.000000
MAV,3.246000e+02,2.908000e+02,2.569600e+02,2.330800e+02,2.193600e+02,200.800000,193.480000,192.360000,181.080000,181.480000,...,173.720000,174.680000,176.480000,173.800000,175.480000,175.520000,174.000000,176.480000,175.160000,174.080000
LOG,1.222418e+01,1.162212e+01,1.100042e+01,1.053669e+01,1.022189e+01,9.836845,9.678618,9.610132,9.379895,9.391816,...,9.225506,9.220000,9.254577,9.226989,9.231412,9.227531,9.225503,9.248681,9.224160,9.230151
WL,2.090000e+03,2.591000e+03,2.398000e+03,2.095000e+03,2.621000e+03,2309.000000,2101.000000,2634.000000,2222.000000,2113.000000,...,1975.000000,2314.000000,2407.000000,1951.000000,2370.000000,2405.000000,1987.000000,2433.000000,2360.000000,1980.000000
ACC,8.360000e+01,1.036400e+02,9.592000e+01,8.380000e+01,1.048400e+02,92.360000,84.040000,105.360000,88.880000,84.520000,...,79.000000,92.560000,96.280000,78.040000,94.800000,96.200000,79.480000,97.320000,94.400000,79.200000
DASDV,9.994790e+01,1.174040e+02,1.115635e+02,1.002665e+02,1.181025e+02,108.339551,100.523836,118.559760,106.201852,101.728192,...,94.003768,106.734874,109.807976,93.839180,109.074516,110.422484,95.540785,111.290348,108.825931,94.723721
ZC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
WAMP,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
features_dfs_renamed = [
    features_df.T.add_prefix(channel+'_')
    for features_df,channel in zip(features_dfs,channels)
]
features_dfs_renamed[0]

,emg0_VAR,emg0_RMS,emg0_IEMG,emg0_MAV,emg0_LOG,emg0_WL,emg0_ACC,emg0_DASDV,emg0_ZC,emg0_WAMP,emg0_MYOP,emg0_FR,emg0_MNP,emg0_TP,emg0_MNF,emg0_MDF,emg0_PKF,emg0_WENT
0,3978.5600,330.671620,8115.0,324.60,12.224179,2090.0,83.60,99.947903,0.0,0.0,0.28,NaN,157333.607850,2.360004e+06,0.063822,0.0,0.0,0.103639
1,4307.8400,298.114877,7270.0,290.80,11.622122,2591.0,103.64,117.404039,0.0,0.0,0.24,NaN,126688.790212,1.900332e+06,0.067338,0.0,0.0,0.138075
2,3712.4384,264.084986,6424.0,256.96,11.000421,2398.0,95.92,111.563510,0.0,0.0,0.12,NaN,98990.915293,1.484864e+06,0.065246,0.0,0.0,0.214809
3,3346.0736,240.150703,5827.0,233.08,10.536694,2095.0,83.80,100.266520,0.0,0.0,0.04,NaN,81310.184274,1.219653e+06,0.063784,0.0,0.0,0.168800
4,3836.8704,227.937886,5484.0,219.36,10.221887,2621.0,104.84,118.102533,0.0,0.0,0.00,NaN,72063.792800,1.080957e+06,0.067877,0.0,0.0,0.215427
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,3317.7696,184.729640,4388.0,175.52,9.227531,2405.0,96.20,110.422484,0.0,0.0,0.00,NaN,46388.460926,6.958269e+05,0.068345,0.0,0.0,0.384310
319,2753.4400,181.740034,4350.0,174.00,9.225503,1987.0,79.48,95.540785,0.0,0.0,0.00,NaN,45481.684043,6.822253e+05,0.063975,0.0,0.0,0.283425
320,3353.3696,185.737880,4412.0,176.48,9.248681,2433.0,97.32,111.290348,0.0,0.0,0.00,NaN,46851.529715,7.027729e+05,0.067828,0.0,0.0,0.305643
321,3244.1344,184.187839,4379.0,175.16,9.224160,2360.0,94.40,108.825931,0.0,0.0,0.00,NaN,46239.284745,6.935893e+05,0.067708,0.0,0.0,0.384993


In [18]:
features = pd.concat(features_dfs_renamed, axis=1)
features

,emg0_VAR,emg0_RMS,emg0_IEMG,emg0_MAV,emg0_LOG,emg0_WL,emg0_ACC,emg0_DASDV,emg0_ZC,emg0_WAMP,...,emg1_ZC,emg1_WAMP,emg1_MYOP,emg1_FR,emg1_MNP,emg1_TP,emg1_MNF,emg1_MDF,emg1_PKF,emg1_WENT
0,3978.5600,330.671620,8115.0,324.60,12.224179,2090.0,83.60,99.947903,0.0,0.0,...,0.0,0.0,0.52,NaN,154665.089386,2.319976e+06,0.063678,0.0,0.0,0.077196
1,4307.8400,298.114877,7270.0,290.80,11.622122,2591.0,103.64,117.404039,0.0,0.0,...,0.0,0.0,0.32,NaN,125130.516086,1.876958e+06,0.067209,0.0,0.0,0.101334
2,3712.4384,264.084986,6424.0,256.96,11.000421,2398.0,95.92,111.563510,0.0,0.0,...,0.0,0.0,0.16,NaN,97905.398121,1.468581e+06,0.065007,0.0,0.0,0.159849
3,3346.0736,240.150703,5827.0,233.08,10.536694,2095.0,83.80,100.266520,0.0,0.0,...,0.0,0.0,0.04,NaN,79594.232302,1.193913e+06,0.064045,0.0,0.0,0.129806
4,3836.8704,227.937886,5484.0,219.36,10.221887,2621.0,104.84,118.102533,0.0,0.0,...,0.0,0.0,0.00,NaN,70816.899286,1.062253e+06,0.067652,0.0,0.0,0.158283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,3317.7696,184.729640,4388.0,175.52,9.227531,2405.0,96.20,110.422484,0.0,0.0,...,0.0,0.0,0.00,NaN,45684.064177,6.852610e+05,0.067736,0.0,0.0,0.281949
319,2753.4400,181.740034,4350.0,174.00,9.225503,1987.0,79.48,95.540785,0.0,0.0,...,0.0,0.0,0.00,NaN,44251.163250,6.637674e+05,0.063890,0.0,0.0,0.221517
320,3353.3696,185.737880,4412.0,176.48,9.248681,2433.0,97.32,111.290348,0.0,0.0,...,0.0,0.0,0.00,NaN,45894.649340,6.884197e+05,0.067517,0.0,0.0,0.229980
321,3244.1344,184.187839,4379.0,175.16,9.224160,2360.0,94.40,108.825931,0.0,0.0,...,0.0,0.0,0.00,NaN,45384.768419,6.807715e+05,0.067108,0.0,0.0,0.286456


In [19]:
record_X = features.values
record_X

array([[3.97856000e+03, 3.30671620e+02, 8.11500000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 7.71963008e-02],
       [4.30784000e+03, 2.98114877e+02, 7.27000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.01333652e-01],
       [3.71243840e+03, 2.64084986e+02, 6.42400000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.59848657e-01],
       ...,
       [3.35336960e+03, 1.85737880e+02, 4.41200000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.29979793e-01],
       [3.24413440e+03, 1.84187839e+02, 4.37900000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.86456191e-01],
       [2.71527360e+03, 1.81711640e+02, 4.35200000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.14049505e-01]])

In [20]:
record_X.shape

(323, 36)

In [21]:
record_Xs = []
record_Xs.extend(record_X)
record_Xs.extend(record_X)
record_Xs
display(len(record_Xs))
display(len(record_Xs[0]))

646

36

In [22]:
X = np.array(record_Xs)
X

array([[3.97856000e+03, 3.30671620e+02, 8.11500000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 7.71963008e-02],
       [4.30784000e+03, 2.98114877e+02, 7.27000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.01333652e-01],
       [3.71243840e+03, 2.64084986e+02, 6.42400000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.59848657e-01],
       ...,
       [3.35336960e+03, 1.85737880e+02, 4.41200000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.29979793e-01],
       [3.24413440e+03, 1.84187839e+02, 4.37900000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.86456191e-01],
       [2.71527360e+03, 1.81711640e+02, 4.35200000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.14049505e-01]])

In [23]:
X.shape

(646, 36)

In [24]:
record_y = [0]*features.shape[0]
len(record_y)

323

In [25]:
record_ys = []
record_ys.extend(record_y)
record_ys.extend(record_y)
len(record_ys)

646

In [26]:
y = np.array(record_ys)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [27]:
y.shape

(646,)

In [34]:
from emg_pattern_recognition.feature_extraction import features_estimation

# sampling frequency 
freq = 10
# sliding window
frame = 25
step = 10

def transform(records):
    record_Xs, record_ys = [],[]
    for target_idx,record in enumerate(records):
        
        features_dfs = []
        
        for channel_name,channel in record.items():
            features_df = features_estimation(np.array(channel), channel_name, freq, frame, step)[0]
            features_df_renamed = features_df.T.add_prefix(channel_name+'_')
            features_dfs.append(features_df_renamed)
        
        features = pd.concat(features_dfs, axis=1)

        
        record_X = features.values
        record_y = [target_idx]*features.shape[0]

        record_Xs.extend(record_X)
        record_ys.extend(record_y)
    
    return np.array(record_Xs), np.array(record_ys)

X,y = transform(records)

C:\Users\LeNoVo\Desktop\31Project\v3\emg_pattern_recognition\feature_extraction.py:169: RuntimeWarning: invalid value encountered in scalar divide
  return ULC / UHC
C:\Users\LeNoVo\miniforge3\envs\myjlabenv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(


EMG features were from channel emg0 extracted successfully
EMG features were from channel emg1 extracted successfully


C:\Users\LeNoVo\Desktop\31Project\v3\emg_pattern_recognition\feature_extraction.py:169: RuntimeWarning: invalid value encountered in scalar divide
  return ULC / UHC
C:\Users\LeNoVo\miniforge3\envs\myjlabenv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\LeNoVo\Desktop\31Project\v3\emg_pattern_recognition\feature_extraction.py:169: RuntimeWarning: invalid value encountered in scalar divide
  return ULC / UHC
C:\Users\LeNoVo\miniforge3\envs\myjlabenv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(


EMG features were from channel emg0 extracted successfully
EMG features were from channel emg1 extracted successfully


C:\Users\LeNoVo\Desktop\31Project\v3\emg_pattern_recognition\feature_extraction.py:169: RuntimeWarning: invalid value encountered in scalar divide
  return ULC / UHC
C:\Users\LeNoVo\miniforge3\envs\myjlabenv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(
C:\Users\LeNoVo\Desktop\31Project\v3\emg_pattern_recognition\feature_extraction.py:169: RuntimeWarning: invalid value encountered in scalar divide
  return ULC / UHC
C:\Users\LeNoVo\miniforge3\envs\myjlabenv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(


EMG features were from channel emg0 extracted successfully
EMG features were from channel emg1 extracted successfully


C:\Users\LeNoVo\Desktop\31Project\v3\emg_pattern_recognition\feature_extraction.py:169: RuntimeWarning: invalid value encountered in scalar divide
  return ULC / UHC
C:\Users\LeNoVo\miniforge3\envs\myjlabenv\Lib\site-packages\pywt\_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(


In [35]:
X

array([[3.97856000e+03, 3.30671620e+02, 8.11500000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 7.71963008e-02],
       [4.30784000e+03, 2.98114877e+02, 7.27000000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.01333652e-01],
       [3.71243840e+03, 2.64084986e+02, 6.42400000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 1.59848657e-01],
       ...,
       [3.73363840e+03, 1.90050625e+02, 4.49900000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.25764108e-01],
       [3.28981440e+03, 1.83057805e+02, 4.34600000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.98782192e-01],
       [3.04495360e+03, 1.84524036e+02, 4.40200000e+03, ...,
        0.00000000e+00, 0.00000000e+00, 2.09536497e-01]])

In [36]:
X.shape

(983, 36)

In [37]:
y.shape

(983,)

In [33]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

model = RandomForestClassifier(criterion='entropy')
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.5959595959595959